In [1]:
%matplotlib inline
%run ../../import_envs.py
print('probtorch:', probtorch.__version__, 
      'torch:', torch.__version__, 
      'cuda:', torch.cuda.is_available())

probtorch: 0.0+5a2c637 torch: 0.4.1 cuda: True


In [2]:
## Load dataset
data_path = "../../../rings_fixed_radius"
Data = torch.from_numpy(np.load(data_path + '/obs.npy')).float()
FIXED_RADIUS = 1.5

NUM_DATASETS, N, D = Data.shape
K = 5 ## number of clusters
SAMPLE_SIZE = 10
NUM_HIDDEN_GLOBAL = 32
NUM_HIDDEN_LOCAL = 64
NUM_STATS = 16

MCMC_SIZE = 10
BATCH_SIZE = 20
NUM_EPOCHS = 500
LEARNING_RATE = 1e-4

CUDA = torch.cuda.is_available()
PATH = 'AG-5rings-%dsteps-%dsamples' % (MCMC_SIZE, SAMPLE_SIZE)
DEVICE = torch.device('cuda:1')

obs_rad = torch.ones(1) * FIXED_RADIUS
noise_sigma = torch.ones(1) * 0.05
if CUDA:
    obs_rad = obs_rad.cuda().to(DEVICE)
    noise_sigma = noise_sigma.cuda().to(DEVICE)
Train_Params = (NUM_EPOCHS, NUM_DATASETS, SAMPLE_SIZE, BATCH_SIZE, CUDA, DEVICE, PATH)
Model_Params = (obs_rad, noise_sigma, N, K, D, MCMC_SIZE)

In [5]:
from local_enc_mu import *
from global_oneshot_mu import *
from global_enc_mu_v1 import *
## if reparameterize continuous variables
Reparameterized = False
# initialization
enc_z = Enc_z(K, D, NUM_HIDDEN_LOCAL, CUDA, DEVICE)
enc_mu = Enc_mu(K, D, NUM_HIDDEN_GLOBAL, NUM_STATS, CUDA, DEVICE, Reparameterized)
oneshot_mu = Oneshot_mu(K, D, NUM_HIDDEN_GLOBAL, NUM_STATS, CUDA, DEVICE, Reparameterized)
if CUDA:
    enc_z.cuda().to(DEVICE)
    enc_mu.cuda().to(DEVICE)
    oneshot_mu.cuda().to(DEVICE)
optimizer =  torch.optim.Adam(list(oneshot_mu.parameters())+list(enc_mu.parameters())+list(enc_z.parameters()),lr=LEARNING_RATE, betas=(0.9, 0.99))
models = (oneshot_mu, enc_mu, enc_z)

In [6]:
# enc_z.load_state_dict(torch.load("../weights/enc-z-%s" % PATH))
# enc_mu.load_state_dict(torch.load("../weights/enc-mu-%s" % PATH))
# oneshot_mu.load_state_dict(torch.load("../weights/oneshot-mu-%s" % PATH))

In [ ]:
from ag_ep import *
train(models, EUBO_fixed_radi, optimizer, Data, Model_Params, Train_Params)

epoch: 0\500 (286s),  symKL_DB_eta: 610272.133,  symKL_DB_z: 1175882.313,  gap_eta: 610239.291,  gap_z: 1175427.469,  loss: -324203.777,  ess: 1.944
epoch: 1\500 (287s),  symKL_DB_eta: 41475.502,  symKL_DB_z: 524788.335,  gap_eta: 41477.705,  gap_z: 524674.737,  loss: -47177.493,  ess: 3.054
epoch: 2\500 (286s),  symKL_DB_eta: 12503.463,  symKL_DB_z: 117742.674,  gap_eta: 12504.476,  gap_z: 117690.386,  loss: -19323.881,  ess: 4.043
epoch: 3\500 (286s),  symKL_DB_eta: 4461.124,  symKL_DB_z: 27963.023,  gap_eta: 4461.881,  gap_z: 27930.502,  loss: -12893.469,  ess: 4.505
epoch: 4\500 (286s),  symKL_DB_eta: 2249.331,  symKL_DB_z: 11077.235,  gap_eta: 2249.635,  gap_z: 11051.890,  loss: -11687.064,  ess: 4.675
epoch: 5\500 (287s),  symKL_DB_eta: 1323.650,  symKL_DB_z: 7812.937,  gap_eta: 1323.559,  gap_z: 7790.745,  loss: -11289.405,  ess: 4.740
epoch: 6\500 (289s),  symKL_DB_eta: 903.397,  symKL_DB_z: 6875.072,  gap_eta: 902.739,  gap_z: 6855.280,  loss: -11155.594,  ess: 4.778
epoch: 7\

In [ ]:
# torch.save(enc_z.state_dict(), "../weights/enc-z-%s" % PATH)
# torch.save(enc_mu.state_dict(), "../weights/enc-mu-%s" % PATH)
# torch.save(oneshot_mu.state_dict(), "../weights/oneshot-mu-%s" % PATH)

In [ ]:
from ag_ep import *
BATCH_SIZE_TEST = 50
Train_Params_Test = (NUM_EPOCHS, NUM_DATASETS, SAMPLE_SIZE, BATCH_SIZE_TEST, CUDA, DEVICE, PATH)

obs, metric_step, reused = test(models, EUBO_fixed_radi, Data, Model_Params, Train_Params_Test)
(q_mu, _, q_z, _) = reused

In [ ]:
%time plot_samples(obs, q_mu, q_z, K, PATH)